# Multiprocessed Sharding

On the previous notebook, we've seen how to leverage Dask/Xarray to shard the data in a multi-threaded fashion.

* We can also perform this with MPI with multiple processed, reading the same HDF5 input file with [the `h5py` library](https://docs.h5py.org/en/stable/quick.html#quick), example below
* If we wanted to perform write concurrency, `h5py` can be compiled with parallel processing in mind with the proper flags. [Follow the guide](https://docs.h5py.org/en/stable/mpi.html).

## Debugging the script

Open the file named `shard_parallel.py` and debug it.

In [1]:
!cat shard_parallel.py

import h5py
from mpi4py import MPI
import numpy as np
import os.path as osp

from kosmoss import PROCESSED_DATA_PATH

def main() -> None:

    # Being multiprocessed, threads can't open the same file for concurrency-related issues
    # So we don't read the timestep variable from the config.yaml file and instead, have to fix the value
    timestep = 1000
    h5_path = osp.join(PROCESSED_DATA_PATH, f'features-{timestep}.h5')
    out_dir = osp.join(PROCESSED_DATA_PATH, f"features-{timestep}")
    os.makedirs(out_dir, exist_ok=True)

    # The MPI Rank uniquely identify each process
    rank = MPI.COMM_WORLD.rank
    print(f'worker of rank {rank} started.')

    # Each process will produce 53 files
    for subidx in np.arange(53):
        
        print(f'processing slice {subidx} for rank {rank}.')
        
        # Each file holding 4800 records
        start = rank * 53 * 2 ** 4 + subidx * 4800
        end = start + 4800

        with h5py.File(h5_path, 'r') as feats:

            # G

## Executing the script

For some reason, `mpiexec` doesn't fancy being called from within a notebook?
You'll have to execute the follow command in a terminal manually from within the script directory.

Run `htop` on one terminal and run the following block in another one:
<code>
mpiexec -n 16 python shard_parallel.py
</code>
This will launch the same Python script on 16 MPI nodes.